**TODO**
* get rid of requests we don't use
* annotate the code to explain what we're doing and why [tutorial!]
* have to explain what broad categories we wanted to analyze and why
    * clean up our doc about what categories we want to use and add it to the github, explain why we are only using certain [un] categories for now (most bang for our buck)

In [12]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import sqlite3


import matplotlib.pyplot as plt
import datetime
import re

In [ ]:
category_links_df = pd.read_csv('category-links.csv')
category_links_df

visited_categories = []
resolution_urls = list()
total_resolutions = 0

In [ ]:
def get_res_links_from_page(page_link, total_resolutions):
    
    links = []
    
    page = requests.get(page_link)
    if page.status_code != 200:
        raise Exception("Something went wrong loading category: " + category + ", error code: " + page.status_code)

    html = page.text
    soup = BeautifulSoup(html, 'html.parser')

    resolution_cnt = int(soup.find("strong", {"class": None}).text)

    if resolution_cnt > 50:
        if resolution_cnt%50 == 0:
            page_count = (int) (resolution_cnt/50)
        else: page_count = int((resolution_cnt/50) + 1)
    else: page_count = 1

    for page in range(page_count):

        for div in soup.find_all("div", {"class": "moreinfo"}):
            res_link_suffix = div.find("a")["href"]
            links.append('https://digitallibrary.un.org' + res_link_suffix)

        time.sleep(10)

        #load the next page if you're not already on the last page (0 indexing)
        if (page+1 != page_count):
            next_page_link_suffix = soup.find("span", {"class": "rec-navigation"}).findAll("a")[-1]["href"]
            next_page_link = 'https://digitallibrary.un.org' + next_page_link_suffix
            #print(page)
            #print(next_page_link)

            page = requests.get(next_page_link)
            if page.status_code != 200:
                raise Exception("Something went wrong loading next page, error code: " + page.status_code)

            html = page.text
            soup = BeautifulSoup(html, 'html.parser')
            
        
    if (total_resolutions + resolution_cnt != len(links)):
        raise Exception("resolution count does not match")
        
    return links

In [1]:
## get unique resolutions => THIS DOES NOT WORK????????????
# len(np.unique(resolution_urls))

#Make resolution_urls unique
def unique(list1):
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

# ex of function use in next line
#unq_res_urls = unique(resolution_urls)

In [ ]:
#get the initial website with the categories
for cat, link in zip (category_link_df["un category"], category_link_df["link"]):
    if (cat not in visited_categories):
        try:
            cat_res_links = get_res_links_from_page(page_link, total_resolutions) #grab the links, may throw an exception
            resolution_urls.extend(cat_res_links) #append the new links list to the bigger old links list
            total_resolutions = len(resolution_urls) #update total resolution count for next runthrough
            visited_categories.append(cat)
        except:
            print("oh no the UN blocked you maybe :(")
        finally:
            time.sleep(3) #delay to hopefully prevent the un from detecting and blocking us


unq_res_urls = unique(resolution_urls)

#write results to file
textfile = open("res_links.txt", "w")
for res_url in unq_res_urls:
    textfile.write(res_url + "\n")
textfile.close()

In [ ]:
#Get vote data from each resolution

#resolution df (x= resolution index, y = name, year)
#country_votes df (row = country, col= resolution index) row,col = vote

for res_url in unq_res_urls:
    res_page = requests.get(res_url)
    if res_page.status_code != 200:
        raise Exception("Something went wrong loading resolution, error code: " + res_page.status_code)
        
    html = res_page.text
    soup = BeautifulSoup(html, 'html.parser')

    metadata = soup.find_all("span", {"class": "value col-xs-12 col-sm-9 col-md-10"})

    #number of metadata sections changes from resolution to resolution, so start counting from bottom for some
    res_title = metadata[0] 
    #res_title.text
    vote_summary = metadata[len(metadata) -4]
    vote_date = metadata[len(metadata) -3]
    vote = metadata[len(metadata) -2]
    time.sleep(10) #can change later
    
    #STORE THIS INFO IN DF EACH TIME

In [19]:
conn = sqlite3.connect("res_data.db")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE voting_data (
        country TEXT PRIMARY KEY,
        resolution_id TEXT
    )""")

OperationalError: table voting_data already exists

In [39]:
res_page = requests.get('https://digitallibrary.un.org/record/671966?ln=en')
if res_page.status_code != 200:
    raise Exception("Something went wrong loading resolution, error code: " + res_page.status_code)

html = res_page.text
soup = BeautifulSoup(html, 'html.parser')

metadata = soup.find("div", {"id" : "details-collapse"})
#metadata

#checking to make sure the vote was recorded
# we only care about recorded votes since they allow us to track how countries change their views over time
row_content_meta = metadata.find_all("span", {"class" : "value col-xs-12 col-sm-9 col-md-10"})
recorded_vote = [False if 'NON-RECORDED' in row.get_text() else True for row in row_content_meta]

rows = metadata.find_all("div", {"class" : "metadata-row"})

title = ""
res_id = ""
date = ""
vote_table = ""

if False not in recorded_vote:

    for row in rows:
        row_title = rows[-2].find("span", {"class" : "title col-xs-12 col-sm-3 col-md-2"}).text
        row_value = rows[-2].find("span", {"class" : "value col-xs-12 col-sm-9 col-md-10"})
        
        #strip newline chars from the string
        row_title = row_title.strip()
        
        if row_title == 'Title':
            title = row_value.text
        elif row_title == 'Resolution':
            res_id = row_value.text
        elif row_title == 'Vote date':
            date = row_value.text
        elif row_title == 'Vote':
            vote_table = row_value
            # save the html to add country individual vote data to voting db
            # iterate through br's to do this more easily

    
    ## now that you have the resolution id, put that as the 
            
row_value

Vote
Vote
Vote
Vote
Vote
Vote
Vote
Vote
Vote
Vote


<span class="value col-xs-12 col-sm-9 col-md-10">Y AFGHANISTAN<br/> N ALBANIA<br/> A ALGERIA<br/> A ARGENTINA<br/> Y AUSTRALIA<br/> Y AUSTRIA<br/> Y BARBADOS<br/> Y BELGIUM<br/> Y BOLIVIA<br/> Y BOTSWANA<br/> A BRAZIL<br/> Y BULGARIA<br/> A BURMA<br/> A BURUNDI<br/> Y BYELORUSSIAN SSR<br/>  CAMBODIA<br/> Y CAMEROON<br/> Y CANADA<br/> A CENTRAL AFRICAN REPUBLIC<br/> Y CEYLON<br/> Y CHAD<br/> Y CHILE<br/> Y CHINA<br/> Y COLOMBIA<br/> A CONGO (BRAZZAVILLE)<br/> Y CONGO (LEOPOLDVILLE)<br/> Y COSTA RICA<br/> N CUBA<br/> Y CYPRUS<br/> Y CZECHOSLOVAKIA<br/> Y DAHOMEY<br/> Y DENMARK<br/>  DOMINICAN REPUBLIC<br/> Y ECUADOR<br/> Y EL SALVADOR<br/> Y ETHIOPIA<br/> Y FINLAND<br/> A FRANCE<br/> A GABON<br/>  GAMBIA<br/> Y GHANA<br/> Y GREECE<br/> Y GUATEMALA<br/> A GUINEA<br/> Y GUYANA<br/>  HAITI<br/> Y HONDURAS<br/> Y HUNGARY<br/> Y ICELAND<br/> A INDIA<br/> Y INDONESIA<br/> Y IRAN<br/> Y IRAQ<br/> Y IRELAND<br/> Y ISRAEL<br/> Y ITALY<br/> Y IVORY COAST<br/> Y JAMAICA<br/> Y JAPAN<br/> Y JORDAN<b

In [18]:
conn.close()